# 02 — Feature Preparation
Mục tiêu: đọc `cleaned.parquet`, kiểm lại leakage, chọn cột, và lưu snapshot modelling-ready `data/processed/dataset_for_clf.parquet`.

In [ ]:
CLEANED_PATH = 'data/processed/cleaned.parquet'
OUTPUT_DATASET_PATH = 'data/processed/dataset_for_clf.parquet'

# (tuỳ chọn) lọc những dòng thiếu target
DROP_ROWS_WITHOUT_TARGET = True


In [ ]:
from pathlib import Path
import pandas as pd

# Find project root (works from notebooks/ or notebooks/runs/)
import os
PROJECT_ROOT = Path(os.getcwd())
while not (PROJECT_ROOT / 'src').exists() and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

cleaned_path = (PROJECT_ROOT / CLEANED_PATH).resolve()
out_path = (PROJECT_ROOT / OUTPUT_DATASET_PATH).resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(cleaned_path)
print('loaded:', cleaned_path)
print('shape:', df.shape)
df.head()

In [ ]:
# Check leakage columns (PM2.5 & pm25_24h phải loại khỏi feature khi train)
leak_cols = [c for c in ['PM2.5', 'pm25_24h'] if c in df.columns]
print('Leakage columns present (OK, but will be excluded in modelling):', leak_cols)

if DROP_ROWS_WITHOUT_TARGET:
    before = len(df)
    df = df[df['aqi_class'].notna()].copy()
    print('dropped rows without target:', before - len(df))


In [ ]:
# Gợi ý feature set (để sinh viên thấy rõ)
drop_cols = {'PM2.5', 'pm25_24h', 'aqi_class', 'datetime'}
feature_cols = [c for c in df.columns if c not in drop_cols]
print('n_features:', len(feature_cols))
feature_cols[:30]

In [ ]:
df.to_parquet(out_path, index=False)
print('Saved:', out_path)